In [1]:
import cv2
import mediapipe as mp
import numpy as np
import pygame
import random

mp_hands = mp.solutions.hands
hands = mp_hands.Hands()
mp_drawing = mp.solutions.drawing_utils

pygame.init()
WIDTH, HEIGHT = 640, 480
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Hand Punch Game")
clock = pygame.time.Clock()

ball_color = (0, 255, 0)
ball_radius = 20
ball_speed = 5
balls = []

score = 0
font = pygame.font.Font(None, 36)

cap = cv2.VideoCapture(0)

def detect_punch(landmarks):
    if landmarks:
        wrist_y = landmarks[0].y
        index_finger_y = landmarks[8].y
        if index_finger_y < wrist_y:  
            return True
    return False

def draw_balls():
    global balls
    for ball in balls:
        pygame.draw.circle(screen, ball_color, (ball[0], ball[1]), ball_radius)
        ball[1] += ball_speed

def update_balls(punch_position):
    global balls, score
    for ball in balls[:]:
        if punch_position and (ball[0] - ball_radius < punch_position[0] < ball[0] + ball_radius) and (ball[1] - ball_radius < punch_position[1] < ball[1] + ball_radius):
            balls.remove(ball)
            score += 1
        elif ball[1] > HEIGHT:
            balls.remove(ball)

running = True
while running and cap.isOpened():
    success, frame = cap.read()
    if not success:
        break

    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    frame = cv2.flip(frame, 1)

    rgb_frame = frame
    results = hands.process(rgb_frame)

    punch_detected = False
    punch_position = None

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            punch_detected = detect_punch(hand_landmarks.landmark)
            if punch_detected:
                punch_position = (int(hand_landmarks.landmark[8].x * WIDTH), int(hand_landmarks.landmark[8].y * HEIGHT))
            mp_drawing.draw_landmarks(rgb_frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

    camera_surface = pygame.surfarray.make_surface(cv2.transpose(rgb_frame))
    
    screen.blit(camera_surface, (0, 0))

    if random.randint(0, 100) < 5: 
        balls.append([random.randint(0, WIDTH), 0])

    draw_balls()
    update_balls(punch_position)

    score_text = font.render(f"Score: {score}", True, (255, 255, 255))
    screen.blit(score_text, (10, 10))

    pygame.display.flip()
    clock.tick(30)

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

cap.release()
pygame.quit()
cv2.destroyAllWindows()


pygame 2.6.0 (SDL 2.28.4, Python 3.9.13)
Hello from the pygame community. https://www.pygame.org/contribute.html


c:\Users\Ahmed Yasser\AppData\Local\Programs\Python\Python39\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
